In [1]:
import random as ran
import numpy as np

def random_probabilities(num=5):
    r = [ran.random() for i in range(num)]
    s = sum(r)
    return np.array([ i/s for i in r ])

def random_matrix(num=5):
    matrix = np.zeros(shape=(5, 5))
    for i in range(num):
        matrix[i] = random_probabilities(num=num)
    return matrix

In [2]:
P = random_matrix()
Q = random_matrix()

In [3]:
from numpy.linalg import norm

def l1_distance(P, Q):
    return np.sum(np.abs(P - Q))

In [4]:
l1_distance(P, Q)

2.5109673888410007

https://stats.stackexchange.com/questions/280885/estimate-the-kullback-leibler-kl-divergence-with-monte-carlo

In [86]:
from math import log

def monte_carlo_distance(P, Q, n_samples_per_cluster=100):
    x, y = P.shape
    assert x == y
    
    total_distance = 0
    for cluster in range(x):
        cum_sum = P[cluster].cumsum()
        for i in range(n_samples_per_cluster):
            next_state = np.where(cum_sum >= ran.random())[0][0]
            
            total_distance += log(P[cluster][next_state] / Q[cluster][next_state])
    return total_distance / (n_samples_per_cluster * x)

def kl_distance2d(P, Q):
    x, y = P.shape
    assert x == y
    
    total_distance = 0
    for cluster in range(x):
        for next_cluster in range(y):
            total_distance += P[cluster][next_cluster] * log(P[cluster][next_cluster] / Q[cluster][next_cluster])
    
    return total_distance / (x * y)

def kl_distance(P, Q):
    total_distance = 0
    for i in range(len(P)):
        if Q[i] == 0 or P[i] == 0:
            continue
        total_distance += P[i] * log(P[i]/Q[i])
            
    return total_distance

def matrix_distance(P, Q, n_samples_per_cluster=100):
     return (monte_carlo_distance(P, Q, n_samples_per_cluster=n_samples_per_cluster) + 
             monte_carlo_distance(Q, P, n_samples_per_cluster=n_samples_per_cluster)) / 2

In [12]:
monte_carlo_distance(P,Q)

0.23447440559159635

In [14]:
A = np.array([[0.3, 0.4, 0.3], [0.2, 0.4, 0.4], [0.8, 0.15, 0.05]])
B = np.array([[0.3, 0.4, 0.3], [0.2, 0.5, 0.3], [0.8, 0.15, 0.05]])

print(monte_carlo_distance(A, B, n_samples_per_cluster=10000))
print(monte_carlo_distance(B, A, n_samples_per_cluster=10000))
print(kl_distance2d(A, B))
print(kl_distance2d(B, A))

0.00776174648497669
0.00788279080951285
0.00286837871723
0.00280746154684


In [15]:
matrix_distance(A, B)

0.013514452431863793

In [16]:
import numpy as np

p1 = np.array([0.5, 0.5])
p2 = np.array([0.5, 0.5])

A1 = np.array([[0.9, 0.1], [0.2, 0.8]])
A2 = np.array([[0.7, 0.3], [0.4, 0.6]])

B1 = np.array([[0.1, 0.3, 0.6], [0.2, 0.1, 0.7]])
B2 = np.array([[0.3, 0.5, 0.2], [0.6, 0.2, 0.2]])

In [17]:
import numpy as np
from scipy.linalg import eig 

def find_stationary(transition_matrix):
    S, U = eig(transition_matrix.T)
    stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
    stationary = stationary / np.sum(stationary)
    return stationary

def kl_hmm(p1, p2, A1, A2, B1, B2):
    stationary = find_stationary(A1)
    return sum([stationary[i] * (kl_distance(A1[i], A2[i]) + kl_distance(B1[i], B2[i])) for i in range(len(stationary))])

In [18]:
kl_hmm(p1, p2, A1, A2, B1, B2)

0.56805785052903368

Other methods

- https://stackoverflow.com/questions/21308848/markov-chain-stationary-distributions-with-scipy-sparse

- def solveStationary( A ):
    """ x = xA where x is the answer
    x - xA = 0
    x( I - A ) = 0 and sum(x) = 1
    """
    n = A.shape[0]
    a = np.eye( n ) - A
    a = np.vstack( (a.T, np.ones( n )) )
    b = np.matrix( [0] * n + [ 1 ] ).T
    return np.linalg.lstsq( a, b )[0]


In [33]:
solveStationary(A1)

array([ 0.66666667,  0.33333333])

In [21]:
from sklearn.externals import joblib

onlineEM = joblib.load('onlineEM_60_500_13_host_specific.pkl')

In [161]:
from math import log


def kl_distance(P, Q):
    total_distance = 0
    for i in range(len(P)):
        if Q[i] == 0 or P[i] == 0:
            continue
        total_distance += P[i] * log(P[i]/Q[i])
            
    return total_distance

def solveStationary(A):
    """ x = xA where x is the answer
    x - xA = 0
    x( I - A ) = 0 and sum(x) = 1
    """
    n = A.shape[0]
    a = np.eye( n ) - A
    a = np.vstack( (a.T, np.ones( n )) )
    b = np.matrix( [0] * n + [ 1 ] ).T
    return np.squeeze(np.asarray(np.linalg.lstsq( a, b )[0]))

def distance(A1, A2):
    stationary = solveStationary(A1)
    return sum([stationary[i] * (2*kl_distance(A1[i], A2[i])) for i in range(len(stationary))])

def hmm_distance(A1, A2):
    return (distance(A1, A2) + distance(A2, A1)) / 2

In [ ]:
j = 1
for host in onlineEM.hosts:
    print(find_stationary(onlineEM.hosts[host]['transitiion_matrix']))
    
    t = onlineEM.hosts[host1]['points_per_cluster'] / 500
    #t = onlineEM.gammas
    for i in range(100):
        t.dot(onlineEM.hosts[host]['transitiion_matrix'])
        
    print(t)
    print()

In [131]:
for host in onlineEM.hosts:
    onlineEM.hosts[host]['stationary'] = find_stationary(onlineEM.hosts[host]['transitiion_matrix']).astype(np.float64)

c:\users\sotiris\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [164]:
host1 = 'C2023'
host2 = 'C4831'

"""
print(onlineEM.hosts[host1]['transitiion_matrix'])
print(onlineEM.hosts[host1]['points_per_cluster'])
print()
print(onlineEM.hosts[host2]['transitiion_matrix'])
print(onlineEM.hosts[host2]['points_per_cluster'])

"""

hmm_distance(onlineEM.hosts[host1]['transitiion_matrix'], onlineEM.hosts[host2]['transitiion_matrix'])

0.03643474670815508

In [158]:
matrices = []

j = 1

for host in onlineEM.hosts:
    matrices.append(onlineEM.hosts[host]['transitiion_matrix'])

    j += 1
    if j == 100:
        break

In [160]:
matrices = np.array(matrices)
means = np.mean(matrices, axis=0)

In [200]:
from numpy.linalg import norm
import random as ran
import numpy as np

class kMeans:
    def __init__(self, em, n_clusters=3, initial_centers=None, n_iter=20, error_tolerance=1e-6):
        if initial_centers is None:
            # TODO
            i = 0
            centers = []
            for host in em.hosts:
                centers.append(em.hosts[host]['transitiion_matrix'])
                
                i += 1
                if i == n_clusters:
                    break
                    
            self.centers = np.array(centers)
        else:
            self.centers = initial_centers

        self.em = em
        self.n_clusters = n_clusters
        self.n_iter = n_iter
        self.error_tolerance = error_tolerance

    def run(self):
        self.clusters = {}
        for _ in range(self.n_iter):
            for host in self.em.hosts:
                distances = np.array([hmm_distance(self.em.hosts[host]['transitiion_matrix'], kmeans.centers[i]) for i in range(self.n_clusters)])
                self.clusters[host] = np.argmin(distances)

            errors = []
            for j, c in enumerate(self.centers):
                members = [k for k,v in self.clusters.items() if v == j]
                
                # find new center
                matrices = []
                for host in members:
                    matrices.append(self.em.hosts[host]['transitiion_matrix'])
                    
                matrices = np.array(matrices)
                new_center = np.mean(matrices, axis=0)
                
                errors.append(hmm_distance(new_center, self.centers[j]))
                self.centers[j] = new_center

            print(max(errors))
            if max(errors) < self.error_tolerance:
                break

    def classify(self, data_point):
        return min(range(self.n_clusters), key=lambda p: dist(data_point, self.centers[p]))

In [201]:
kmeans = kMeans(onlineEM)

In [202]:
kmeans.run()

0.263572590047
0.150034079414
0.154249200967
0.216474964528
0.139242263872
0.308896009143
0.0401578878139
0.172836566462
0.0279414693369
0.0897869479462
0.111003693049
0.104635605425
0.126524535967
0.112626688531
0.181063924269
0.0633143292344
0.0685160201086
0.117064362112
0.0825585884921
0.127070809268


In [209]:
from math import log
import scipy.stats.distributions


def poisson(x, l):
    return_value = 1
    for x_i, l_i in zip(x, l):
        return_value *= scipy.stats.distributions.poisson.pmf(x_i, l_i)
    return return_value

def calculate_likelihood_em(data):
    # first reset previous point for all hosts for rerun

    previous_points = {}
    for host in onlineEM.hosts:
        previous_points[host] = onlineEM.hosts[host]['hard_previous']

    total_likelihood = []
    
    i = 0
    for point in data:
        i += 1
        if i % 5000 == 0:
            print(i)
            
        host = point[-1]

        previous_point = previous_points[host]

        point_center = onlineEM.closest_centers([point])
        closest_center = np.argmax(point_center)

        previous_points[host] = closest_center
        
        probabilities = kmeans.centers[kmeans.clusters[host]]
    
        participation = probabilities * np.array([poisson(point, lambda_i) for lambda_i in onlineEM.lambdas])
        
        likelihood = log(np.sum(participation))
    
        total_likelihood.append(likelihood)

    return sum(total_likelihood) / len(total_likelihood)

In [212]:
from sklearn.externals import joblib

groupped_data1 = joblib.load('groupped_data1_60_500.pkl')

calculate_likelihood_em(groupped_data1.values[:10000,[0,1,3]])

5000
10000


-2.0344317582041054

For the first 10000 3 clusters find and simple mean the log likelihood is: -2.0344317582041054